In [1]:
import sys
sys.path.append('../')
import os, json, time, random
from google_scholar import scrape_google_scholar_profile, save_google_scholar_profile, scrape_affiliates
from utils import create_driver
from tqdm import tqdm
import pandas as pd

In [2]:
raw_dr_ntu_dir='./raw_dr_ntu'
raw_faculty_db = 'scse_profile'

process_faculty_db_dir ='./processed'
process_faculty_db = 'scse_profile'
process_co_author_db = 'google_scholar_co_author'

raw_google_scholar_dir = './raw_google_scholar'

In [3]:
faculties = pd.read_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'))
google_scholar_faculties = faculties[faculties['google_scholar'].notna()]

In [4]:
# create co author db
co_author_db = {'full_name': [], 'google_scholar_id': [], 'google_scholar': []}

for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, url, id = row['full_name'], row['google_scholar'], row['google_scholar_id']
    
    if os.path.exists(f'{raw_google_scholar_dir}/{id}.json'):
        with open(f'{raw_google_scholar_dir}/{id}.json', 'r') as f:
            raw_google_scholar_profile = json.load(f)
        
        co_authors = raw_google_scholar_profile['co_authors']

        for co_author in co_authors:
            co_author_link = co_author['link']
            co_author_id = co_author['link'].split('user=')[1].split('&')[0]
            co_author_name = co_author['name']
            
            if co_author_id not in co_author_db['google_scholar_id']:
                co_author_db['google_scholar'].append(co_author_link)
                co_author_db['google_scholar_id'].append(co_author_id)
                co_author_db['full_name'].append(co_author_name)

co_author_db = pd.DataFrame(co_author_db)
co_author_db.to_csv(f'{process_faculty_db_dir}/{process_co_author_db}.csv', index=False)
len(co_author_db)

0it [00:00, ?it/s]

74it [00:00, 959.90it/s]


2282

In [ ]:
driver = create_driver(debug=False)

for i, row in tqdm(co_author_db.iterrows()):
    url, id = row['google_scholar'], row['google_scholar_id']

    co_author_profile = scrape_google_scholar_profile(url, driver)
    co_author_profile['last_updated'] = datetime.datetime.now().strftime("%Y/%m/%d %H:%M")
    save_google_scholar_profile(co_author_profile, output_filename=f'{id}.json', output_dir=raw_google_scholar_dir)

    if random.uniform(0,1)>0.8:
        time.sleep(random.uniform(5,7))

    if random.uniform(0,1)>0.9:
        driver.quit()
        driver = create_driver(debug=False)

driver.quit()